In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader


########### Loading the dataset #############################
data_normal = pd.read_csv("data_normal.csv")

class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length


X_train, X_test, y_train, y_test = train_test_split(np.array(data_normal[["x_1", "x_2"]]), 
                                                    np.array(data_normal["y"]), test_size=0.3, random_state=42)

trainset = dataset(X_train, y_train)
testset = dataset(X_test, y_test)

#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)
testloader = DataLoader(testset,batch_size=64,shuffle=False)

################## Define the model ###########################
class LinearModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.sig = nn.Sigmoid()
        # self.relu = nn.ReLU()
                           
    def get_weights(self):
        return self.weight
    
    def forward(self,x):
        out = self.fc1(x)
        # out = self.relu(out)
        out = self.sig(self.fc2(out)) #sigmoid as we use BCELoss
        return out

######################### Hyper-parameters #########################
input_size = 2
hidden_size = 32
num_classes = 1 
num_epochs = 5
learning_rate = 0.001
BATCH_SIZE_1 = 1

model = LinearModel(input_size, hidden_size, num_classes)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


def train(model, train_loader, optimizer):
    model.train()
    y_true = []
    y_pred = []
    for i in tqdm(train_loader):
        
        data, target = i
 
        # data, target = data.cuda(), target.cuda()
       
        #FORWARD PASS
        output = model(data.float())
        loss = criterion(output, target.unsqueeze(1)) 
        
        #BACKWARD AND OPTIMIZE
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # PREDICTIONS 
        pred = np.round(output.detach())
        target = np.round(target.detach())             
        y_pred.extend(pred.tolist())
        y_true.extend(target.tolist())
        
    print("Accuracy on training set is" ,         
    accuracy_score(y_true,y_pred))


#TESTING THE MODEL
def test(model, test_loader):
    #model in eval mode skips Dropout etc
    model.eval()
    y_true = []
    y_pred = []
    
    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for i in test_loader:
            
            #LOAD THE DATA IN A BATCH
            data,target = i
            
            
            # the model on the data
            output = model(data.float())
                       
            #PREDICTIONS
            pred = np.round(output)
            target = target.float()
            y_true.extend(target.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
    
            
    print("Accuracy on test set is" , accuracy_score(y_true,y_pred))
    print("***********************************************************")


train(model, trainloader, optimizer)
test(model, testloader)

100%|██████████| 197/197 [00:00<00:00, 471.09it/s]


Accuracy on training set is 0.49412698412698414
Accuracy on test set is 0.4961111111111111
***********************************************************
